In [9]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [10]:
#hypothetical OG data
df_hypothetical_OG = pd.read_csv('../report/TriTrypDB-65_All_species_clean_Ortholog_Group_Full_of_hypotetical_boolean.tsv', sep='\t')

#RBH results
df_rbh = pd.read_csv('../../results/reciprocal_best_hit_SingleOrgApproach_TSV/rbh_all_in_one_file.tsv', sep='\t', index_col=0)

#model organims annotation data
df_MO_annotation = pd.read_csv('../../results/Gene_annotation_info_from_uniprot_model_spp.tsv', sep='\t', low_memory=False, index_col=0)


#estructure to genID to OG

df_OG_genID_uniprot  = pd.read_csv('../report/TriTrypDB-65_All_species_clean_ortholog_groups_x_sequence_clustering_x_UNIPROT.tsv', sep='\t')




#this are the db that i want to use as reference of annotation plus some usefull info. But removing not relevant columns for this analysis.
columns_for_df =  ['Entry','Entry Name','Protein names', 'Organism', 'Organism (ID)', 'Proteomes','Taxonomic lineage' ,'EC number', 'Protein families', 'OrthoDB','PANTHER', 'InterPro', 'Pfam', 'eggNOG', 'Gene Ontology (cellular component)'] #'KEGG'


#removing low annotation proteins from model org
df_MO_annotation = df_MO_annotation[columns_for_df].dropna(thresh=10)

#keeping only relevant columns
df_kineto_annotation = df_kineto_annotation[columns_for_df]

In [11]:

#aca uso inner porque quiero evitar las proteinas sin buena anotacion en el resultado
df_rbh_km = df_rbh.merge(df_MO_annotation, left_on='target_uniprot_accession', right_on='Entry', how='left', suffixes=['_kineto', '_model'])
print(df_rbh_km.shape)


# adding cluster representer
df_rbh_km = df_rbh_km.merge(df_OG_genID_uniprot, left_on='query_uniprot_accession', right_on='uniprot', how='left')
print(df_rbh_km.shape)

df_rbh_km = df_rbh_km.merge(df_hypothetical_OG, left_on='Ortholog_Group', right_on='cluster_representer_or_OG', how='left')
print(df_rbh_km.shape)


(168148, 33)
(168150, 37)
(168150, 41)


In [28]:
df_rbh_km = (
    df_rbh_km
    .sort_values('target', ascending=False)
    .drop_duplicates(subset=['query_uniprot_accession', 'target_uniprot_accession']
                    )
)

In [26]:
df_rbh_km_tophits =(
    df_rbh_km
    .sort_values(['evalue'])
    .groupby('query')
    .head(5)
)

In [30]:
df_rbh_km_tophits

,query,target,alnlen,fident,evalue,qstart,qend,qlen,tstart,tend,tlen,aligmentinfo,proteome,spp,query_uniprot_accession,target_uniprot_accession,COV_query,COV_target,Entry,Entry Name,Protein names,Organism,Organism (ID),Proteomes,Taxonomic lineage,EC number,Protein families,OrthoDB,PANTHER,InterPro,Pfam,eggNOG,Gene Ontology (cellular component),Ortholog_Group,Gene ID,uniprot,pLDDT_mean,cluster_representer_or_OG,OGsize,OGhypotetical_proteins,only_hypothetical_OG
98356,AF-wheelerlab_Tb927.10.6050_1_1703_1-F1-model_...,AF-P29742-F1-model_v4.pdb.gz,3904,0.377,0.00,0,1702,1703,2,1676,1678,16M2I28M1I11M1D8M1I68M2I16M1I72M1I33M3D152M1D1...,UP000000803,DROME,wheelerlab_Tb927.10.6050_1_1703_1,P29742,1.00,1.00,P29742,CLH_DROME,Clathrin heavy chain,Drosophila melanogaster (Fruit fly),7227.0,UP000000803: Chromosome X,"melanogaster subgroup (no rank), melanogaster ...",NaN,Clathrin heavy chain family,5474327at2759;,PTHR10292:SF1;PTHR10292;,IPR016024;IPR000547;IPR015348;IPR016025;IPR022...,PF00637;PF09268;PF13838;PF01394;,KOG0985;,cell cortex [GO:0005938]; clathrin coat of coa...,TM35_000023460,Tb927.10.6050,wheelerlab_Tb927.10.6050_1_1703_1,83.0,TM35_000023460,56,5.0,False
8855,AF-A0A061J2Y6-F1-model_v4.pdb,AF-Q5A0N3-F1-model_v4.pdb.gz,4729,0.579,0.00,5,794,795,0,797,854,161M5I300M2D117M1I176M4I9M3D3M13D14M,UP000000559,CANAL,A0A061J2Y6,Q5A0N3,0.99,0.93,Q5A0N3,Q5A0N3_CANAL,Ribonucleoside-diphosphate reductase (EC 1.17....,Candida albicans (strain SC5314 / ATCC MYA-287...,237561.0,UP000000559: Chromosome 2,"Candida albicans (species), Candida (genus), C...",1.17.4.1,Ribonucleoside diphosphate reductase large cha...,5472715at2759;,PTHR11573;PTHR11573:SF6;,IPR005144;IPR013346;IPR000788;IPR013509;IPR008...,PF03477;PF02867;PF00317;,KOG1112;,ribonucleoside-diphosphate reductase complex [...,TcIL3000_0_44200,TRSC58_03619,A0A061J2Y6,93.0,TcIL3000_0_44200,64,6.0,False
123947,AF-A4HVH9-F1-model_v4.pdb,AF-E9PZJ8-F1-model_v4.cif.gz,7740,0.398,0.00,150,2166,2167,153,2193,2198,43M4D9M21I31M5D10M4I5M6I36M6I6M11D17M25D32M1D9...,UP000000589,MOUSE,A4HVH9,E9PZJ8,0.93,0.93,E9PZJ8,ASCC3_MOUSE,Activating signal cointegrator 1 complex subun...,Mus musculus (Mouse),10090.0,UP000000589: Chromosome 10,"Mus (subgenus), Mus (genus), Murinae (subfamil...",3.6.4.12,Helicase family,57056at2759;,PTHR47961;PTHR47961:SF13;,IPR003593;IPR035892;IPR011545;IPR014001;IPR001...,PF00270;PF00271;PF02889;,KOG0952;,activating signal cointegrator 1 complex [GO:0...,PCON_0046850,LINF_130009900,A4HVH9,78.0,PCON_0046850,116,9.0,False
27740,AF-A4HVH9-F1-model_v4.pdb,AF-K7KW51-F1-model_v4.cif.gz,7705,0.404,0.00,140,2146,2167,143,2079,2088,23M5D27M2I11M16I7M1I28M1D9M4I5M6I69M17D5M6I8M4...,UP000008827,SOYBN,A4HVH9,K7KW51,0.93,0.93,K7KW51,K7KW51_SOYBN,DExH-box ATP-dependent RNA helicase DExH14,Glycine max (Soybean) (Glycine hispida),3847.0,UP000008827: Chromosome 6,"Glycine subgen. Soja (subgenus), Glycine (genu...",NaN,NaN,57056at2759;,PTHR47961;PTHR47961:SF13;,IPR003593;IPR035892;IPR011545;IPR014001;IPR001...,PF00270;PF00271;PF02889;,KOG0952;,nucleus [GO:0005634]; spliceosomal complex [GO...,PCON_0046850,LINF_130009900,A4HVH9,78.0,PCON_0046850,116,9.0,False
55917,AF-A0A504XKS0-F1-model_v4.pdb,AF-A0A3Q0KEX5-F1-model_v4.pdb.gz,5351,0.432,0.00,1,1063,1067,25,1153,1267,256M19I33M1D190M1I18M1I155M2D9M2D7M2D56M2D45M1...,UP000008854,SCHMA,A0A504XKS0,A0A3Q0KEX5,1.00,0.89,A0A3Q0KEX5,A0A3Q0KEX5_SCHMA,isoleucine--tRNA ligase (EC 6.1.1.5) (Isoleucy...,Schistosoma mansoni (Blood fluke),6183.0,UP000008854: Unassembled WGS sequence,"Schistosoma (genus), Schistosomatidae (family)...",6.1.1.5,Class-I aminoacyl-tRNA synthetase family,NaN,PTHR42780:SF1;PTHR42780;,IPR001412;IPR002300;IPR033709;IPR002301;IPR023...,PF08264;PF00133;,NaN,NaN,LSM04_008747,LdBPK_365870.1,A0A504XKS0,90.0,LSM04_008747,71,4.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32471,AF-A0A7J6XPY0-F1-model_v4.pdb,AF-A